# Mail Address and Password to access pinecone:
- mail address: nlp.pinecone@gmail.com
- password: UniHeidelberg
- https://www.pinecone.io

# API env and key for Pinecone:
- environment: gcp-starter
- key: 1218c885-67e3-492f-b1ab-215405569e97

In [9]:
%pip install pinecone-client --quiet
%pip install langchain_pinecone --quiet
%pip install langchain --quiet
%pip install transformers --quiet
%pip install sentence-transformers --quiet
%pip install tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.6 MB/s eta 0:00:00


In [11]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
#import langchain_pinecone
import pinecone
import json
from langchain_community.document_loaders import JSONLoader
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


## Load data and convert to langchain_core.documents.base.Document

The CSVs were first loaded as Pandas DataFrame objects to discard the column 'Abstract_Length' column, followed by the conversion of the data into langchain_core.documents.base.Document format

In [4]:
df1 = pd.read_csv("processed_data_part1.csv")
df2 = pd.read_csv("processed_data_part2.csv")
df = pd.concat([df1, df2], ignore_index=True)
df.drop("Abstract_Length", axis=1, inplace=True)
print(f"df shape: {df.shape}")
df.head()

df shape: (58535, 6)


,PMID,Title,Abstract,Authors,Publication Date,DOI
0,24645995,α-1 antitrypsin and chronic fatigue syndrome: ...,SUMMARY Several lines of evidence support the ...,Jose Alegre; Sandra Camprubi; Ana GarciaQuintana,2013-Mar,10.2217/pmt.12.84
1,24565439,A data-driven acute inflammation therapy.,Acute inflammation is a severe medical conditi...,Vladan Radosavljevic; Kosta Ristovski; Zoran O...,2013,10.1186/1755-8794-6-S3-S7
2,24505723,Voxelwise spectral diffusional connectivity an...,Human brain connectivity can be studied using ...,Junning Li; Yan Jin; Yonggang Shi; Ivo D Dinov...,2013,10.1007/978-3-642-40811-3_82
3,24472488,Systems integrity in health and aging - an ani...,Human lifespan is positively correlated with c...,Marije Oostindjer; Gro V Amdam,2013-Jan-07,10.1186/2046-2395-2-2
4,24460364,Multi-agent systems: effective approach for ca...,Physicians in order to study the causes of can...,Niloofar Mohammadzadeh; Reza Safdari; Azin Rahimi,2013,10.7314/apjcp.2013.14.12.7757


In [5]:
loader = DataFrameLoader(df, page_content_column="Abstract")
docs_all = loader.load()
print(f"len: {len(docs_all)}")
docs_all[0]

len: 58535


Document(page_content='SUMMARY Several lines of evidence support the involvement of inflammatory and immunologic abnormalities in chronic fatigue syndrome CFS Since recent studies have shown that α1 antitrypsin AAT possesses antiinflammatory properties the potential therapeutic effect of AAT treatment on CFS has been investigated A 49yearold woman diagnosed with CFS was treated with intravenous infusions of a human plasmaderived AAT concentrate 60 mgkg body weight weekly for 8 consecutive weeks The patients monocyte elastase a regulator of inflammatory processes was 1170 Umg At completion of treatment improvement in maximal workload was observed 540717 of predicted Additionally amelioration in working memory scores 8394 and perceptual organization scores 7583 were detected on the Wechsler Adult Intelligence ScaleIII test Monocyte elastase decreased to a normal range 150 Umg Improvement in functional capacity allowed the patient to work in parttime employment These findings suggest a po

In [6]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=400, chunk_overlap=100)
chunked_docs = text_splitter.split_documents(docs_all)
len(chunked_docs)

62615

In [7]:
chunked_docs[0]

Document(page_content='SUMMARY Several lines of evidence support the involvement of inflammatory and immunologic abnormalities in chronic fatigue syndrome CFS Since recent studies have shown that α1 antitrypsin AAT possesses antiinflammatory properties the potential therapeutic effect of AAT treatment on CFS has been investigated A 49yearold woman diagnosed with CFS was treated with intravenous infusions of a human plasmaderived AAT concentrate 60 mgkg body weight weekly for 8 consecutive weeks The patients monocyte elastase a regulator of inflammatory processes was 1170 Umg At completion of treatment improvement in maximal workload was observed 540717 of predicted Additionally amelioration in working memory scores 8394 and perceptual organization scores 7583 were detected on the Wechsler Adult Intelligence ScaleIII test Monocyte elastase decreased to a normal range 150 Umg Improvement in functional capacity allowed the patient to work in parttime employment These findings suggest a po

In [12]:
# Saving chunked_docs to pickle file
with open('chunked_docs.pkl', 'wb') as file:  # Note the 'wb' mode for writing binary files
    pickle.dump(chunked_docs, file)

## Initilize embedding model

In [7]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_built() else "cpu"
embedding_model_name = 'thenlper/gte-base'
print(f"device: {device}")

device: cuda


In [10]:
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}
embed_model = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
embed_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='thenlper/gte-base', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

## Initialize Pinecone

**Please do not run the sections below, (which could cause to deletion of the index or the addition of more data to the index,) if you have not made any modifications to your PubMed data or if you don't decide to use a different embedding**

In [13]:
index_name = "pubmed"
embedding_dim = 768
os.environ["PINECONE_API_KEY"]="1218c885-67e3-492f-b1ab-215405569e97"

In [14]:
pc = pinecone.Pinecone()
pc.list_indexes()
# pc.delete_index(name=index_name)

{'indexes': []}

In [15]:
pc.create_index(name=index_name, dimension=embedding_dim,   spec=pinecone.PodSpec(environment="gcp-starter"), metric='cosine')
pc.list_indexes()

{'indexes': [{'dimension': 768,
              'host': 'pubmed-pf44p5o.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'pubmed',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

## Create embeddings and load on pinecone using langchain

In [16]:
docsearch = langchain_pinecone.Pinecone.from_documents(chunked_docs, embed_model, index_name=index_name)

In [17]:
pc.Index(index_name).describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.62615,
 'namespaces': {'': {'vector_count': 62615}},
 'total_vector_count': 62615}

## Testing

In [27]:
# retrieved documents are logical
query = "Is there a cure to cancer?"
docs = docsearch.similarity_search(query, k=3)
for i, d in enumerate(docs):
    print(f"\n## Document {i+1}\n")
    print(d.page_content)


## Document 1

The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from cancer every year indicating that the current treatment strategy is not leading to a cancer cure but to a cancer remission that is the temporary absence of manifestations of a particular disease after months or years of remission in most patients cancer will inevita

In [32]:
# retrieved documents are logical
retriever = docsearch.as_retriever(search_type="mmr")
matched_docs = retriever.get_relevant_documents(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i+1}\n")
    print(d.page_content)


## Document 1

The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from cancer every year indicating that the current treatment strategy is not leading to a cancer cure but to a cancer remission that is the temporary absence of manifestations of a particular disease after months or years of remission in most patients cancer will inevita

In [33]:
# retrieved documents are logical
found_docs = docsearch.max_marginal_relevance_search(query, k=3, fetch_k=10)
for i, d in enumerate(found_docs):
    print(f"\n## Document {i+1}\n")
    print(d.page_content)


## Document 1

The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from cancer every year indicating that the current treatment strategy is not leading to a cancer cure but to a cancer remission that is the temporary absence of manifestations of a particular disease after months or years of remission in most patients cancer will inevita